# Utiliser l'extension [Pydentic](https://pypi.org/project/pydentic/)

L'extension [pydentic](https://docs.pydantic.dev/usage/models/) est une extension qui aide à la gestion de données sous forme de classe d'objets.

Nous commençons par un [exemple simple](https://techtutorialsx.com/2022/06/03/pydantic-getting-started/). Nous déclarons une classe `Person` qui hérite du modèle de base. On se donne une entrée sous forme d'un dictionnaire. La commande `Person(**data)` permet de transférer le dictionaire de données dans l'objet (sans se soucier de l'ordre dans le dictionaire).

Si on entrait une erreur, elle serait interceptée et affichée. Enfin, l'affichage de l'objet est amélioré : on affiche son contenu au lieu de donner le
 pointeur.

In [21]:
from pydantic import BaseModel, ValidationError

class Person(BaseModel):
    age: int
    name: str
    is_married: bool

data = {
    'name': "John",
    'age': 30,
    'is_married': False
}

try:
    person = Person(**data)
    print(person.dict())
except ValidationError as e:
    print(e)


{'age': 30, 'name': '34.5', 'is_married': False}


Voici un autre exemple contenant un stucture un peu plus complexe.

In [5]:
from typing import List
from pydantic import BaseModel, ValidationError
 
class Address(BaseModel):
    street: str
    building: int
 
class Person(BaseModel):
    age: int
    name: str
    is_married: bool
    address: Address
    languages: List[str]
 
 
data = {
    'age': 10,
    'name': 'John',
    'is_married': False,
    'address': {
        'street': 'st street',
        'building': 10
    },
    'languages':['pt-pt', 'en-us']
}
 
try:
    person = Person(**data)
    print(person.dict())
 
except ValidationError as e:
    print("Exception as str:")
    print(e)
    print("Exception as json:")
    print(e.json())

{'age': 10, 'name': 'John', 'is_married': False, 'address': {'street': 'st street', 'building': 10}, 'languages': ['pt-pt', 'en-us']}


On voit que malgré la structure à plusieurs niveau de `data`, l'instruction synthétique utilisée pour le constructeur `Person(**data)` fonctionne toujours.

## Exemple avec des fichiers json

L'intérêt de ces objets contenant des données est de les relier à des fichiers json, comme le montre le [tutoriel vidéo de Arjan code]()

In [6]:
import json

def main() -> None:
    """Main function."""

    # read from a JSON file
    with open("./data.json") as file:
        data = json.load(file)
        print(data[0]["title"])

if __name__ == "__main__":
    main()


Zero to one


Si nous faisons dériver notre classe du modèle de base de [Pydentic](https://docs.pydantic.dev/usage/models/) nous héritons des fonctionalités de ce modèle.

In [6]:
import json
import pydantic
from typing import Optional

class Book(pydantic.BaseModel):
    title: str
    author: str
    publisher: str
    price: float
    isbn_10: Optional[str]
    subtitle: Optional[str]


def main() -> None:
    """Main function."""

    # read from a JSON file
    with open("./data.json") as file:
        data = json.load(file)
        books = [Book(**item) for item in data]
        print(books[0].title)

if __name__ == "__main__":
    main()


Zero to one


La méthode utilisée pour construire le collection d'objets à partir du fichier `json` mérite quelques explications : la collection `books` est une liste d'objet, définie ici en extension en utilisant le constructeur `Book()`. L'argument de ce constructeur est ici un dictionaire anonyme, qui correspond à chaque dictionaire lu dans la liste de dictionaires `data`. C'est une expression à retenir, car très classique pour récupérer ainsi une liste d'objet depuis un fichier `json`.

https://betterprogramming.pub/the-beginners-guide-to-pydantic-ba33b26cde89

# Ajouter des validations de données

On peut ajouter des validations propres à notre modèle de données.

https://www.youtube.com/watch?v=g-F3FubxHd0

In [4]:
from pydantic import BaseModel, validator
import datetime
from enum import Enum

class Level(Enum):
    BEGINNER = 1
    INTERMEDIATE = 2
    ADVANCED = 3

class Student(BaseModel):
    first_name: str
    last_name: str
    age: int
    date_joined: datetime.date
    level: Level

    @validator("age")
    def validate_age(cls, age):
        if age < 10:
            raise ValueError("Age must be 10 or above")
        return(age)

    @validator("level")
    def validate_level_from_age(cls, level, values):
        if level is Level.ADVANCED and age < 14:
            raise ValueError("To be advanced the student must be above 13!")
        return level

try:
    student = Student(
        first_name="Harry",
        last_name="Potter",
        age=8,
        date_joined=datetime.date(2020,1,1),
        level=Level.BEGINNER
    )
    print(student)
    
except ValueError as e:
    print(e) 


1 validation error for Student
age
  Age must be 10 or above (type=value_error)


On pourra faire plusieurs tests pour voir le fonctionnement.